In [1]:
import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from encode_utils.new_flatten_lattice import get_dictlist
from encode_utils.new_mask_utils import randomsingle, useall
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')


2022-12-24 06:50:44.727927: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-24 06:50:44.727950: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# setup / load model
encodemod = get_effrerank_model("noun")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_44/checkpoints/epoch=9-step=40000.ckpt.


freeze embeds


In [3]:
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [5]:
base = "outputs/graph_pickles/frtest_reversed/"
noun_explode = pd.read_csv("outputs/score_csvs/nounlargeexplodev1.csv")

In [5]:
# Issue #3 - there's a difference between gold selected with standard re-rank and with EEL

In [9]:
# get token level scores from model
def get_hyp_sco(inphyp, posids=None):
    
    tokens = xlm_tok(inphyp, return_tensors='pt').to(device)
    tokens = tokens.input_ids
    if posids is None: 
        positionids = None
    else:
        # get token at the end
        positionids = torch.tensor(posids+[posids[-1]+1]).unsqueeze(0).to(device)
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(device)

    toked_inp = xlm_tok(["noun"], return_tensors="pt").to(device)
    predout = encodemod(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    tmppred = predout['score']
    #norm = predout['norm']
    return tmppred

# TODO do a validation that old score generation way and current have same bests
def get_ind_result(ind):
    graph = pickle.load(open(base+str(ind), 'rb'))
    texplode = noun_explode[noun_explode['ref']==graph['ref']].reset_index()
    # recalculate noun scores for all
    cscos = []
    for t in list(texplode['hyp']):
        hs = get_hyp_sco(t)
        cscos.append(torch.sum(get_hyp_sco(t)))#*(hs.shape[1]-1))
    print("scodist - ", [float(f) for f in cscos])
    print("max - ", float(max(cscos)))
    bestpath , flattened, pnodes, mask, sents, posids, pred, _, \
            flnodes, dpath, beplist, besclist, totnodes, bsco = run_comstyle(graph, encodemod, default_scofunct, "noun", {'afunc':randomsingle}, True)
    predhyp = bestpath[0][4:]
    ph = get_hyp_sco(predhyp)
    predsco = torch.sum(ph)#*(ph.shape[1]-1)
    print("pred - ", float(predsco))
    print("predhyp")
    return mask, sents, posids, pred, list(texplode['hyp'])

In [32]:
msk, snts, pids, scos, hyps = get_ind_result(18)

scodist -  [0.37691840529441833, 0.3711412250995636, 0.7022441625595093, 0.33818960189819336, 0.3423137068748474, 0.3410169780254364, 0.674796462059021, 0.342754065990448, 0.6870383024215698, 0.6777890920639038, 0.36100950837135315, 0.3775254487991333]
max -  0.7022441625595093
pred -  0.7022441625595093
predhyp


In [47]:
hyps[-2]

'To some extent, China is going to face obstacles on its way, and there is no guarantee that its leaders will admit their failures or adjust their strategy accordingly.'

In [8]:
posmat = pids.expand(len(snts[0]), len(snts[0])).to("cpu")
posver = posmat*msk
inpmat = snts.expand(len(snts[0]), len(snts[0])).to("cpu")
inpmat = inpmat*(msk>0)
scomat = scos.squeeze(-1).expand(len(snts[0]), len(snts[0])).to("cpu")
scomat = scomat*(msk>0)


In [33]:
pids

tensor([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
         20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
         10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
         28, 29, 30, 31, 32, 33, 34, 35, 36, 37,  4,  5,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
         30, 31, 32, 33, 34, 35, 36, 37, 38, 11, 12, 13, 14, 15, 16, 17, 18, 19,
         20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
         38,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,  9, 10, 11, 12, 13, 14,
         15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
         33, 34, 35, 36, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
         29, 30, 31, 32, 33, 34, 35, 36,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
         13, 14, 15, 16, 17,

In [36]:
scomat[35][:37]*torch.max(pids).to("cpu")

tensor([0.0000, 0.0315, 0.0320, 0.0334, 0.3809, 0.0520, 0.9946, 0.2352, 0.2196,
        0.1561, 1.0815, 0.3335, 0.4159, 0.3077, 1.5724, 0.1403, 0.4810, 0.4012,
        0.4049, 0.5259, 2.2846, 0.6981, 0.4920, 2.6411, 0.7156, 0.7587, 0.6133,
        3.0344, 0.8132, 0.7931, 1.2603, 0.4955, 3.8419, 1.0233, 0.5695, 0.3316,
        0.0000])

In [44]:
(get_hyp_sco(hyps[11])*36).shape

torch.Size([1, 37, 1])

In [22]:
xlm_tok.decode(inpmat[35])

'<s> To a certain extent, China will encounter obstacles on its way, and there is no guarantee that its leaders will admit their failures or adjust their strategy accordingly.<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s>

In [11]:
for i in range(len(posver)):
    for j in range(1, len(posver[0])):
        val = posver[i][j]
        if val == 0:
            continue
        if val <= posver[i][j-1]:
            print(i)

In [43]:
# mask has no recomb
assert torch.sum(msk!=torch.tril(msk))==0
# no weird posids

In [5]:
# Issue #2 - graph may be getting corrupted during the process of conversion, make sure identical exploded even after conversion
mb_tok = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

# recursive method to sanity check first (assume no recomb, no cycles)
def explode_path(ind, processed=False):
    graph = pickle.load(open(base+str(ind), 'rb'))
    allpaths = []
    if processed:
        fldict, flnodes = get_dictlist(graph, True)
        explode_helper([], flnodes[0], allpaths, xlm_tok)
    else:
        explode_helper([], graph['root'], allpaths, mb_tok)
        allpaths = [a[10:] for a in allpaths]
    nexplode = noun_explode[noun_explode['ref']==graph['ref']].reset_index()
    return allpaths, list(nexplode['hyp'])

# helper for exploding paths
def explode_helper(prevpath, node, apaths, tok):
    prevpath.append(node.token_idx)
    if len(node.nextlist)==0:
        #print(prevpath)
        apaths.append(mb_tok.decode(prevpath))
    else:
        for n in node.nextlist:
            explode_helper(prevpath, n, apaths, tok)
    prevpath.pop()

In [ ]:
apths = []
explode_helper([], pnodes[0][0], apths, xlm_tok)

In [4]:
# Issue #1 - changing pos-ids with identical strings changes token scores for all things
# score check (at token level) the best option, and the new option (should be same until they get to non-identical tokens)

In [8]:
# get token level scores from model
def get_hyp_sco_verb(inphyp, posids=None):
    
    tokens = xlm_tok(inphyp, return_tensors='pt').to(device)
    tokens = tokens.input_ids

    if posids is None: 
        # have to add 2 because of how xlm roberta works
        positionids = torch.arange(len(tokens[0])).unsqueeze(0).to(device)+2
        print(positionids)

    elif -1 in posids:
        positionids = None
    else:
        # get token at the end
        positionids = posids
        #torch.tensor(posids+[posids[-1]+1]).unsqueeze(0).to(device)
        print(positionids)

    # get a causal mask to use
    tmpmask = torch.tril(torch.ones(len(tokens[0]), len(tokens[0]))).unsqueeze(0).to(device)
    
    # this is a dummy string (using CQE model architecture which requires an "input" pass)
    toked_inp = xlm_tok(["noun"], return_tensors="pt").to(device)
    
    # run inputs through the model
    predout = encodemod(toked_inp.input_ids, toked_inp.attention_mask, tokens, positionids, \
        tmpmask)
    
    tmppred = predout['score']
    return tmppred, tokens, positionids, tmpmask


In [16]:
# get scores for identical strings, with different posids
sco_default = get_hyp_sco_verb("I am a bat")
modpos = sco_default[2]
# change last position id
modpos[0][4] = 7
sco_mod = get_hyp_sco_verb("I am a bat", modpos)

tensor([[2, 3, 4, 5, 6, 7]], device='cuda:2')
tensor([[2, 3, 4, 5, 7, 7]], device='cuda:2')


In [18]:
# these two are actually fine. There is some sort of issue in no-recomb cases
print(sco_default[0])
print(sco_mod[0])

tensor([[[0.0000],
         [0.0042],
         [0.0042],
         [0.0043],
         [0.0508],
         [0.0214]]], device='cuda:2', grad_fn=<DivBackward0>)
tensor([[[0.0000],
         [0.0042],
         [0.0042],
         [0.0043],
         [0.0557],
         [0.0211]]], device='cuda:2', grad_fn=<DivBackward0>)
